In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc

# Patch for working directory
path = Path.cwd()

# Get the files from the path provided in the OP
files = Path(path).glob('*.csv')  # .rglob to get subdirectories

In [5]:
print(path)

C:\Users\Patrick_Waldenhofer\Dropbox\UNI\Semester6\Bachelor_Thesis\Practical_Part\Three_Classes


In [3]:
#Find all csv files in directory and concatenate them
dfs = list()
names = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    names.append(f.stem)
    dfs.append(data)
    

df_raw = pd.concat(dfs, ignore_index=True)

In [4]:
#drop last column taht shows csv filename source:
del df_raw[df_raw.columns[-1]]

#df_raw

R1-PA1:VH     R1-PM1:V   R1-PA2:VH     R1-PM2:V   R1-PA3:VH  \
0      70.399324  127673.0908  -49.572308  127648.0176 -169.578319   
1      73.688102  130280.7109  -46.300719  130255.6377 -166.278082   
2      73.733939  130305.7842  -46.254883  130280.7109 -166.232245   
3      74.083443  130581.5902  -45.899649  130556.5169 -165.882741   
4      74.553268  131083.0556  -45.424094  131057.9823 -165.424375   
...          ...          ...         ...          ...         ...   
78372 -58.934439  130706.9565 -178.900342  130681.8832   61.094490   
78373 -59.020382  131057.9823 -178.992015  131032.9090   61.002816   
78374 -59.031842  131083.0556 -179.003474  131032.9090   60.997087   
78375 -59.077678  131108.1288 -179.049311  131057.9823   60.951250   
78376 -59.203729  131158.2754 -179.169632  131108.1288   60.825200   

          R1-PM3:V  R1-PA4:IH   R1-PM4:I   R1-PA5:IH   R1-PM5:I  ...  \
0      127723.2374  65.689611  605.91099  -57.003571  626.78553  ...   
1      130355.9307  71.831719  483.59351  -50.947407  500.98896  ...   
2      130381.0040  71.808800  483.59351  -50.913030  500.98896  ...   
3      130656.8100  72.152575  482.86107  -50.437475  499.15786  ...   
4      131158.2754  72.118198  484.50906  -50.013486  497.69298  ...   
...            ...        ...        ...         ...        ...  ...   
78372  130782.1763 -60.149109  444.59108  178.808669  449.90127  ...   
78373  131108.1288 -61.507019  451.73237  177.857559  454.66213  ...   
78374  131133.2021 -61.535667  451.91548  177.857559  454.66213  ...   
78375  131158.2754 -61.839335  453.38036  177.725778  455.02835  ...   
78376  131208.4219 -62.446670  455.39457  177.336167  456.12701  ...   

       control_panel_log4  relay1_log  relay2_log  relay3_log  relay4_log  \
0                       0           0           0           0           0   
1                       0           0           0           0           0   
2                       0           0           0           0           0   
3                       0           0           0           0           0   
4                       0           0           0           0           0   
...                   ...         ...         ...         ...         ...   
78372                   0           0           0           0           0   
78373                   0           0           0           0           0   
78374                   0           0           0           0           0   
78375                   0           0           0           0           0   
78376                   0           0           0           0           0   

       snort_log1  snort_log2  snort_log3  snort_log4    marker  
0               0           0           0           0  NoEvents  
1               0           0           0           0  NoEvents  
2               0           0           0           0  NoEvents  
3               0           0           0           0  NoEvents  
4               0           0           0           0  NoEvents  
...           ...         ...         ...         ...       ...  
78372           0           0           0           0   Natural  
78373           0           0           0           0   Natural  
78374           0           0           0           0   Natural  
78375           0           0           0           0   Natural  
78376           0           0           0           0   Natural  

[78377 rows x 129 columns]

In [4]:
print(df_raw.shape)

df_raw['marker'].value_counts()

(78377, 129)


Attack      55663
Natural     18309
NoEvents     4405
Name: marker, dtype: int64

In [5]:
#----> UNBALANCED DATA

In [6]:
#df_raw.info()

In [7]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
#show_dtypes(df_raw)

In [8]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [9]:
def marker_to_numeric(x):
        if x=='NoEvents': return 0
        if x=='Natural':   return 1
        if x=='Attack': return 2

df['marker'] = df['marker'].apply(marker_to_numeric)
#df

In [10]:
np.where(df.values >= np.finfo(np.float64).max)

(array([  738,   739,   753, ..., 78352, 78353, 78353], dtype=int64),
 array([26, 26, 26, ..., 55, 26, 55], dtype=int64))

In [11]:
np.isnan(df.values.any())

False

In [12]:
# Replace infinite updated data with nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

(72073, 129)


2    51445
1    16223
0     4405
Name: marker, dtype: int64

In [13]:
# Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful

In [17]:
# UPSAMPLING:

from sklearn.utils import resample,shuffle

df_minority_NoEvents = df[df['marker'] == 0]
df_minority_Natural = df[df['marker'] == 1]
df_not_minority = df[df['marker'] == 2]

#upsample the minority classes
df_minority_NoEvents_upsampled = resample(df_minority_NoEvents,random_state=42,n_samples=51445,replace=True)
df_minority_Natural_upsampled = resample(df_minority_Natural,random_state=42,n_samples=51445,replace=True)

print(df_minority_NoEvents_upsampled.shape)
print(df_minority_Natural_upsampled.shape)
print(df_not_minority.shape)

#concatenate the upsampled dataframes
df_upsampled = pd.concat([df_minority_NoEvents_upsampled, df_minority_Natural_upsampled, df_not_minority ])
#df_upsampled

(51445, 129)
(51445, 129)
(51445, 129)


In [18]:
df_upsampled['marker'].value_counts()

0    51445
1    51445
2    51445
Name: marker, dtype: int64

In [19]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

#This only copies a Series Object, no pandas Dataframe!
df_marker = df_upsampled["marker"].copy()
#Converting to Pandas Dataframe
df_marker = pd.DataFrame(df_marker)

#Delete Marker column because "marker" should not be normalized
del df_upsampled[df_upsampled.columns[-1]]

#Initialize SCaler
scaler = StandardScaler()
#Scale data
data_normalized = scaler.fit_transform(df_upsampled)
#insert normalized data in former pandas frame
df_normalized_nomark = pd.DataFrame(data_normalized, columns=df_upsampled.columns)

#concat "marker" column with normalized rest dataframe, index of marker frame is overwritten
df_normalized = pd.concat([df_normalized_nomark, df_marker.set_index(df_normalized_nomark.index)], axis=1)

#print(df_normalized_nomark)

#print(isinstance(df, pd.DataFrame))

#print(isinstance(df_marker, pd.DataFrame))

#print(df_normalized)


In [20]:
df_normalized.iloc[:,128]

0         0
1         0
2         0
3         0
4         0
         ..
154330    2
154331    2
154332    2
154333    2
154334    2
Name: marker, Length: 154335, dtype: int64

In [23]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [24]:
device = T.device("cpu")

In [21]:
# splitting first time into 70% train and 30% test 
train_df, test_df = train_test_split(df_normalized, test_size=0.3, random_state=42)

# splitting second time 30% test into 15% test and 15% valid overall
test_df, valid_df = train_test_split(test_df, test_size=0.5, random_state=42)

train_df.shape, test_df.shape, valid_df.shape

((108034, 129), (23150, 129), (23151, 129))

In [25]:
# Splitting up features and target columns and convert to numpy array

train_X_np = train_df.iloc[:, : -1].to_numpy() # Select all columns except the last one [: all rows , : -1 all columns except the last one selected]

train_y_np = train_df.iloc[:, -1].to_numpy() # Select the last column [: all rows, -1 only the last column is selected]

valid_X_np = valid_df.iloc[:, : -1].to_numpy() 

valid_y_np = valid_df.iloc[:, -1].to_numpy() 

test_X_np = test_df.iloc[:, : -1].to_numpy() 

test_y_np = test_df.iloc[:, -1].to_numpy() 

# Convert to torch tensors

train_X_tensor = T.tensor(train_X_np, dtype=T.float32)

train_y_tensor = T.tensor(train_y_np, dtype=T.long)

valid_X_tensor = T.tensor(valid_X_np, dtype=T.float32)

valid_y_tensor = T.tensor(valid_y_np, dtype=T.long)

test_X_tensor = T.tensor(test_X_np, dtype=T.float32)

test_y_tensor = T.tensor(test_y_np, dtype=T.long)

In [42]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import pytorch_lightning as pl
from pytorch_lightning import Trainer



In [49]:
# Define the TabNetClassifier model
model = TabNetClassifier(
    n_d=32,
    n_a=32,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    cat_dims=[],
    cat_emb_dim=[],
    optimizer_fn=T.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type='entmax',
    scheduler_params=dict(mode='min', patience=5, min_lr=1e-5, factor=0.5,),
    scheduler_fn=T.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=1,
)

# Train the model
model.fit(
    X_train=train_X_np,
    y_train=train_y_np,
    eval_set=[(valid_X_np, valid_y_np)],
    max_epochs=100,
    patience=10,
    batch_size=32,
    virtual_batch_size=16,
    num_workers=0,
)

# Predict on new data
new_data = torch.randn(10, 10)
predictions = model.predict(new_data.numpy())

# Convert the predicted class probabilities to labels
labels_predicted = torch.argmax(torch.tensor(predictions), dim=1)
print(labels_predicted)

epoch 0  | loss: 0.79696 | val_0_accuracy: 0.51825 |  0:01:45s
epoch 1  | loss: 0.71848 | val_0_accuracy: 0.43558 |  0:03:36s
epoch 2  | loss: 0.70024 | val_0_accuracy: 0.38046 |  0:05:25s
epoch 3  | loss: 0.69192 | val_0_accuracy: 0.48477 |  0:07:19s
epoch 4  | loss: 0.68576 | val_0_accuracy: 0.36076 |  0:09:16s
epoch 5  | loss: 0.68082 | val_0_accuracy: 0.46188 |  0:11:12s
epoch 6  | loss: 0.67501 | val_0_accuracy: 0.34914 |  0:13:09s
epoch 7  | loss: 0.67552 | val_0_accuracy: 0.34219 |  0:15:11s
epoch 8  | loss: 0.69999 | val_0_accuracy: 0.35195 |  0:17:20s
epoch 9  | loss: 0.68448 | val_0_accuracy: 0.33333 |  0:19:27s
epoch 10 | loss: 0.68727 | val_0_accuracy: 0.33774 |  0:21:33s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_accuracy = 0.51825


C:\Users\Patrick_Waldenhofer\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


NameError: name 'torch' is not defined

In [43]:
base_model = TabNetClassifier()


In [44]:
# Step 3: Create PyTorch Lightning module
class TransferLearningModel(pl.LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.BCEWithLogitsLoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.BCEWithLogitsLoss()(y_hat, y)
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.0001)
        return optimizer


In [47]:
# Step 4: Train and evaluate model using PyTorch Lightning Trainer

train_dataset = T.utils.data.TensorDataset(train_X_tensor, train_y_tensor)
val_dataset = T.utils.data.TensorDataset(valid_X_tensor, valid_y_tensor )
model = TransferLearningModel(base_model)
trainer = pl.Trainer( max_epochs=10)
trainer.fit(model, train_dataset, val_dataset)
trainer.test()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: C:\Users\Patrick_Waldenhofer\Dropbox\UNI\Semester6\Bachelor_Thesis\Practical_Part\Three_Classes\lightning_logs


ValueError: optimizer got an empty parameter list

In [30]:
class MyDataset(Dataset):
    def __init__(self, feature_tensor, target_tensor):
        self.feature_tensor = feature_tensor
        self.target_tensor = target_tensor
        
    def __len__(self):
        return len(self.feature_tensor)
    
    def __getitem__(self, idx):
        features = self.feature_tensor[idx]
        target = self.target_tensor[idx]
        
        return features, target

In [31]:
class ThreeClassesDataset(T.utils.data.Dataset):
    
    def __init__(self, src_file):
        all_xy_np = np.array(src_file.values, dtype=np.float64)
        
        tmp_x_np = all_xy_np[:,0:128]
        
        tmp_y_np = all_xy_np[:,128] 
        
        self.x_data = T.from_numpy(tmp_x_np).to(device)
        
        self.y_data = T.from_numpy(tmp_y_np).to(device)
        
    def __len__(self):
        return(self.x_data)
    
    def __getitem__(self,idx):
        preds = self.x_data[idx]
        targets = self.y_data[idx]
        return preds, targets # as a tuple
    

In [32]:
class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(128,10)
        self.hid2 = T.nn.Linear(10,10)
        self.output = T.nn.Linear(10,3)
        
        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.output.weight)
        T.nn.init.zeros_(self.output.bias)
        
    def forward(self, x):
        z = T.tanh(self.hid1(x))
        z = T.tanh(self.hid2(z))
        z = T.log_softmax(self.output(z), dim=1) # NLLLoss() 
        return z        

In [41]:
train_ds = MyDataset(train_X_tensor, train_y_tensor)

test_ds = MyDataset(test_X_tensor, test_y_tensor)



bat_size = 1000

train_loader = DataLoader(train_ds, batch_size = bat_size, shuffle=True, drop_last = True)
#shuffle = True is extremely important because training the networks fails throtugh oscillationg biases and weights
#drop_last = True is good practice because the batch sizes should be evenly divided




In [34]:
# create Network
print("Creating 128-(10-10)-3 neural network ")
net = Net().to(device)
net.train()

Creating 128-(10-10)-3 neural network 


Net(
  (hid1): Linear(in_features=128, out_features=10, bias=True)
  (hid2): Linear(in_features=10, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=3, bias=True)
)

In [35]:
 # 3. train model
max_epochs = 1000
ep_log_interval = 100
lrn_rate = 0.01

loss_func = T.nn.NLLLoss()  # assumes log_softmax()
optimizer = T.optim.SGD(net.parameters(),lr=lrn_rate)

print("bat_size = %3d " % bat_size)
print("loss = " + str(loss_func))
print("optimizer = SGD")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.3f " % lrn_rate)
print("Starting training ")
for epoch in range(0, max_epochs):
    # T.manual_seed(epoch+1)  # reproducibility
    epoch_loss = 0  # for one full epoch

    for (batch_idx, batch) in enumerate(train_loader):
        X = batch[0]  # inputs
        Y = batch[1]  # correct class/label/politics

        optimizer.zero_grad()
        oupt = net(X)
        loss_val = loss_func(oupt, Y)  # a tensor
        epoch_loss += loss_val.item()  # accumulate
        loss_val.backward()
        optimizer.step()

    if epoch % ep_log_interval == 0:
        print("epoch = %5d  |  loss = %10.4f" % \
        (epoch, epoch_loss))
print("Training done ")

bat_size = 1000 
loss = NLLLoss()
optimizer = SGD
max_epochs = 1000 
lrn_rate = 0.010 
Starting training 
epoch =     0  |  loss =    52.2421
epoch =   100  |  loss =    32.1699
epoch =   200  |  loss =    31.2364
epoch =   300  |  loss =    30.3744
epoch =   400  |  loss =    29.2606
epoch =   500  |  loss =    28.7306
epoch =   600  |  loss =    28.3875
epoch =   700  |  loss =    28.1379
epoch =   800  |  loss =    27.9375
epoch =   900  |  loss =    27.7480
Training done 


In [37]:
def accuracy(model, ds):
    # assumes model.eval()
    # item-by-item version
    n_correct = 0; n_wrong = 0
    for i in range(len(ds)):
        X = ds[i][0].reshape(1,-1)  # make it a batch
        Y = ds[i][1].reshape(1)  # 0 1 or 2, 1D
        with T.no_grad():
            oupt = model(X)  # logits form

        big_idx = T.argmax(oupt)  # 0 or 1 or 2
        if big_idx == Y:
            n_correct += 1
        else:
            n_wrong += 1

    acc = (n_correct * 1.0) / (n_correct + n_wrong)
    return acc

In [42]:
# 4. evaluate model accuracy
print("Computing model accuracy ")
net.eval()
acc_train = accuracy(net, train_ds)  # item-by-item
print("Accuracy on training data = %0.4f" % acc_train)
acc_test = accuracy(net, test_ds) 
print("Accuracy on test data = %0.4f" % acc_test)

Computing model accuracy 
Accuracy on training data = 0.7371
Accuracy on test data = 0.7370
